In [15]:
from PyPDF2 import PdfReader
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain_community.callbacks.manager import get_openai_callback
from langchain.text_splitter import RecursiveCharacterTextSplitter # 切分chunk
from langchain_community.embeddings import DashScopeEmbeddings # embedding模型
from langchain_community.vectorstores import FAISS # 向量数据库
from typing import List, Tuple
import os
import pickle

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
if not DASHSCOPE_API_KEY:
    raise ValueError("请设置环境变量 DASHSCOPE_API_KEY")

def extract_text_with_page_numbers(pdf) -> Tuple[str, List[int]]:
    """
    从PDF中提取文本并记录每行文本对应的页码
    
    参数:
        pdf: PDF文件对象
    
    返回:
        text: 提取的文本内容
        page_numbers: 每行文本对应的页码列表
    """
    text = ""
    page_numbers = []

    for page_number, page in enumerate(pdf.pages, start=1):
        extracted_text = page.extract_text()
        if extracted_text:
            text += extracted_text
            page_numbers.extend([page_number] * len(extracted_text.split("\n")))

    return text, page_numbers

def process_text_with_splitter(text: str, page_numbers: List[int], save_path: str = None) -> FAISS:
    """
    处理文本并创建向量存储
    
    参数:
        text: 提取的文本内容
        page_numbers: 每行文本对应的页码列表
        save_path: 可选，保存向量数据库的路径
    
    返回:
        knowledgeBase: 基于FAISS的向量存储对象
    """
    # 创建文本分割器，用于将长文本分割成小块
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " ", ""],
        chunk_size=1000,
        chunk_overlap=200, # 不会因为切分，会造成某些信息放到了上面，没有放到下面，造成的遗漏
        length_function=len,
    )

    # 分割文本
    chunks = text_splitter.split_text(text)
    print(f"文本被分割成 {len(chunks)} 个块。")
        
    # 创建嵌入模型
    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1",
        dashscope_api_key=DASHSCOPE_API_KEY,
    )
    
    # 从文本块创建知识库
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    print("已从文本块创建知识库。")
    
    # 改进：存储每个文本块对应的页码信息
    # 创建原始文本的行列表和对应的页码列表
    lines = text.split("\n")
    
    # 为每个chunk找到最匹配的页码
    page_info = {}
    for chunk in chunks:
        # 查找chunk在原始文本中的开始位置
        start_idx = text.find(chunk[:100])  # 使用chunk的前100个字符作为定位点
        if start_idx == -1:
            # 如果找不到精确匹配，则使用模糊匹配
            for i, line in enumerate(lines):
                if chunk.startswith(line[:min(50, len(line))]):
                    start_idx = i
                    break
            
            # 如果仍然找不到，尝试另一种匹配方式
            if start_idx == -1:
                for i, line in enumerate(lines):
                    if line and line in chunk:
                        start_idx = text.find(line)
                        break
        
        # 如果找到了起始位置，确定对应的页码
        if start_idx != -1:
            # 计算这个位置对应原文中的哪一行
            line_count = text[:start_idx].count("\n")
            # 确保不超出页码列表长度
            if line_count < len(page_numbers):
                page_info[chunk] = page_numbers[line_count]
            else:
                # 如果超出范围，使用最后一个页码
                page_info[chunk] = page_numbers[-1] if page_numbers else 1
        else:
            # 如果无法匹配，使用默认页码5（这里应该根据实际情况设置一个合理的默认值）
            page_info[chunk] = 5
    
    knowledgeBase.page_info = page_info
    
    # 如果提供了保存路径，则保存向量数据库和页码信息
    if save_path:
        # 确保目录存在
        os.makedirs(save_path, exist_ok=True)
        
        # 保存FAISS向量数据库
        knowledgeBase.save_local(save_path)
        print(f"向量数据库已保存到: {save_path}")
        
        # 保存页码信息到同一目录
        with open(os.path.join(save_path, "page_info.pkl"), "wb") as f:
            pickle.dump(page_info, f)
        print(f"页码信息已保存到: {os.path.join(save_path, 'page_info.pkl')}")

    return knowledgeBase

def load_knowledge_base(load_path: str, embeddings = None) -> FAISS:
    """
    从磁盘加载向量数据库和页码信息
    
    参数:
        load_path: 向量数据库的保存路径
        embeddings: 可选，嵌入模型。如果为None，将创建一个新的DashScopeEmbeddings实例
    
    返回:
        knowledgeBase: 加载的FAISS向量数据库对象
    """
    # 如果没有提供嵌入模型，则创建一个新的
    if embeddings is None:
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=DASHSCOPE_API_KEY,
        )
    
    # 加载FAISS向量数据库，添加allow_dangerous_deserialization=True参数以允许反序列化
    knowledgeBase = FAISS.load_local(load_path, embeddings, allow_dangerous_deserialization=True)
    print(f"向量数据库已从 {load_path} 加载。")
    
    # 加载页码信息
    page_info_path = os.path.join(load_path, "page_info.pkl")
    if os.path.exists(page_info_path):
        with open(page_info_path, "rb") as f:
            page_info = pickle.load(f)
        knowledgeBase.page_info = page_info
        print("页码信息已加载。")
    else:
        print("警告: 未找到页码信息文件。")
    
    return knowledgeBase

# 读取PDF文件
pdf_reader = PdfReader('./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf')
# 提取文本和页码信息
text, page_numbers = extract_text_with_page_numbers(pdf_reader)
text

'百度文库  - 好好学习，天天向上  \n-1 上海浦东发展银行西安分行  \n个金客户经理管理考核暂行办法  \n \n \n第一章  总   则 \n第一条   为保证我分行个金客户经理制的顺利实施，有效调动个\n金客户经理的积极性，促进个金业务快速、稳定地发展，根据总行《上\n海浦东发展银行个人金融营销体系建设方案（试行）》要求，特制定\n《上海浦东发展银行西安分行个金客户经理管理考核暂行办法（试\n行）》（以下简称本办法）。  \n第二条   个金客户经理系指各支行（营业部）从事个人金融产品\n营销与市场开拓，为我行个人客户提供综合银行服务的我行市场人\n员。 \n第三条   考核内容分为二大类， 即个人业绩考核、 工作质量考核。\n个人业绩包括个人资产业务、负债业务、卡业务。工作质量指个人业\n务的资产质量。  \n第四条   为规范激励规则，客户经理的技术职务和薪资实行每年\n考核浮动。客户经理的奖金实行每季度考核浮动，即客户经理按其考\n核内容得分与行员等级结合，享受对应的行员等级待遇。  \n 百度文库  - 好好学习，天天向上  \n-2 第二章  职位设置与职责  \n第五条   个金客户经理职位设置为：客户经理助理、客户经理、\n高级客户经理、资深客户经理。  \n第六条   个金客户经理的基本职责：  \n（一）   客户开发。研究客户信息、联系与选择客户、与客户建\n立相互依存、相互支持的业务往来关系，扩大业务资源，创造良好业\n绩； \n（二）业务创新与产品营销。把握市场竞争变化方向，开展市场\n与客户需 求的调研，对业务产品及服务进行创新；设计客户需求的产\n品组合、制订和实施市场营销方案；  \n（三）客户服务。负责我行各类表内外授信业务及中间业务的受\n理和运作，进行综合性、整体性的客户服务；  \n（四）防范风险，提高收益。提升风险防范意识及能力，提高经\n营产品质量；  \n（五）培养人材。在提高自身综合素质的同时，发扬团队精神，\n培养后备业务骨干。  \n 百度文库  - 好好学习，天天向上  \n-3 第三章  基础素质要求  \n第七条   个金客户经理准入条件：  \n（一）工作经历：须具备大专以上学历，至少二年以上银行工作\n经验。  \n（二）工作能力：熟悉我行的各项业务，了解市场情况，熟悉各\n类客户的金融需求

In [12]:
#page_numbers

In [16]:
print(f"提取的文本长度: {len(text)} 个字符。")
    
# 处理文本并创建知识库，同时保存到磁盘
save_dir = "./vector_db"
knowledgeBase = process_text_with_splitter(text, page_numbers, save_path=save_dir)

提取的文本长度: 3881 个字符。
文本被分割成 5 个块。
已从文本块创建知识库。
向量数据库已保存到: ./vector_db
页码信息已保存到: ./vector_db\page_info.pkl


In [17]:
#page_numbers

In [22]:
from langchain_community.llms import Tongyi
llm = Tongyi(model_name="deepseek-v3", dashscope_api_key=DASHSCOPE_API_KEY) # qwen-turbo

# 设置查询问题
#query = "客户经理被投诉了，投诉一次扣多少分"
#query = "客户经理每年评聘申报时间是怎样的？"
query = "How many points are deducted per complaint against a customer manager?"
if query:
    # 执行相似度搜索，找到与查询相关的文档
    docs = knowledgeBase.similarity_search(query,k=2)

    # 加载问答链
    chain = load_qa_chain(llm, chain_type="stuff")

    # 准备输入数据
    input_data = {"input_documents": docs, "question": query}

    # 使用回调函数跟踪API调用成本
    with get_openai_callback() as cost:
        # 执行问答链
        response = chain.invoke(input=input_data)
        print(f"查询已处理。成本: {cost}")
        print(response["output_text"])
        print("来源:")

    # 记录唯一的页码
    unique_pages = set()

    # 显示每个文档块的来源页码
    for doc in docs:
        text_content = getattr(doc, "page_content", "")
        source_page = knowledgeBase.page_info.get(
            text_content.strip(), "未知"
        )

        if source_page not in unique_pages:
            unique_pages.add(source_page)
            print(f"文本块页码: {source_page}")

查询已处理。成本: Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0
According to the context provided, 2 points are deducted for each customer complaint against a customer manager. Specifically, the document states: "客户服务效率低，态度生硬或不及时为客户提供维护服务,有客户投诉的,每投诉一次扣 2分" which translates to "For low customer service efficiency, being rude or failing to provide timely maintenance services to customers, with customer complaints, 2 points will be deducted for each complaint."
来源:
文本块页码: 4
文本块页码: 7


In [23]:
docs

[Document(id='52f02819-66fb-4eec-a40e-240c6659be28', metadata={}, page_content='5.超出最低考核标准可相互折算，折算标准： 50万储蓄 =50万个贷 =50张有效卡 =5分（折算以 5分为单位）  \n \n 百度文库  - 好好学习，天天向上  \n-5 第五章  工作质量考核标准  \n第九条   工作质量考核实行扣分制。工作质量指个金客户经理在\n从事所有个人业务时出现投诉、差错及风险。该项考核最多扣 50分，\n如发生重大差错事故，按分行有关制度处理。  \n（一）服务质量考核：   \n1、工作责任心不强，缺乏配合协作精神；扣 5分 \n2、客户服务效率低，态度生硬或不及时为客户提供维护服务，\n有客户投诉的 ,每投诉一次扣 2分 \n3、不服从支行工作安排，不认真参加分（支）行宣传活动的，\n每次扣 2分； \n4、未能及时参加分行（支行）组织的各种业务培训、考试和专\n题活动的每次扣 2分； \n5、未按规定要求进行贷前调查、贷后检查工作的，每笔扣 5分； \n6、未建立信贷台帐资料及档案的每笔扣 5分； \n7、在工作中有不廉洁自律情况的每发现一次扣 50分。 \n（二）个人资产质量考核：  \n当季考核收息率 97%以上为合格，每降 1个百分点扣 2分；不\n良资产零为合格，每超一个个百分点扣 1分。 \nA.发生跨月逾期，单笔不超过 10万元，当季收回者，扣 1分。 \nB.发生跨月逾期， 2笔以上累计金额不超过 20万元，当季收回\n者，扣 2分；累计超过 20万元以上的，扣 4分。 百度文库  - 好好学习，天天向上  \n-6 C.发生逾期超过 3个月，无论金额大小和笔数，扣 10分。 \n \n第六章  聘任考核程序  \n第十条   凡达到本办法第三章规定的该技术职务所要求的行内职\n工，都可向分行人力资源部申报个金客户经理评聘。  \n第十一条   每年一月份为客户经理评聘的申报时间，由分行人力\n资源部、个人业务部每年二月份组织统一的资格考试。考试合格者由\n分行颁发个金客户经理资格证书，其有效期为一年。  \n第十二条   客户经理聘任实行开放式、浮动制，即：本人申报  —\n— 所在部门推荐  —— 分行考核  —— 行长聘任  —— 每年考评\n调

In [24]:
test = {"statusCode": 1, "statusMessage": "请求成功", "data": {"Id": "10512e4b97c0eda03f0ba93b4eb6dd04", "CompanyName": "青岛百年香传实业有限公司", "CompanyType": "有限责任公司(自然人独资)", "LegalPerson": "李 诗耀", "RegCapital": "1000.000000万", "companyTypeTags": ["独资企业", "有限责任公司"], "CompanyCode": "370220230030416", "CompanyAddress": "山东省青岛市市南区山东路6号甲1号楼3901室", "BusinessScope": "许可项目：食品销售；道路货物运输（ 不含危险货物）。（依法须经批准的项目，经相关部门批准后方可开展经营活动，具体经营项目以相关部门批准文件或许可证件为准）一般项目：货物进出口；办公用品销售；第一类医疗器械销售；教学用模型及教具销售；汽车零配件零售；互联网销售（除销售需要许可的商品）；无船承运业务；技术进出口；进出口代理；国内货物运输代理；物联网应用服务；装卸搬运；信息咨询服务（不含许可类信息咨询服务）；机械设备研发。（除依法须经批准的项目外，凭营业执照依法自主开展经营活动）", "Authority": "青岛市市南区 市场监督管理局", "CompanyStatus": "在营（开业）企业", "EstablishDate": "2014-11-12 00:00:00", "CreditNo": "91370220321511725F", "Province": "山东省", "OperationStartDate": "2014-11-12 00:00:00", "OperationEndDate": None, "IssueDate": "2024-07-18 00:00:00", "ProvinceCode": "370000", "City": "青岛市", "CityCode": "370200", "District": "市南区", "DistrictCode": "370202", "CompanyStatusNew": "正常", "LegalPersonType": 1, "Industry": "批发业", "socialStaffNum": 8, "TaxCode": None, "OrgCode": "321511725", "RealCapital": "900.000000万元", "HistoryNameList": [], "HistoryNames": "", "RegCapitalCurrency": "人民币", "RevokeDate": None, "RevokeReason": None, "CancelDate": None, "CancelReason": None, "CompanyPersonNum": 8, "industryAll": {"L1Name": "批发和零售业", "L2Name": "批发业", "L3Name": "其他批发业", "L4Name": "其他未列 明批发业"}}, "orderNo": 982293803941507072}
test

{'statusCode': 1,
 'statusMessage': '请求成功',
 'data': {'Id': '10512e4b97c0eda03f0ba93b4eb6dd04',
  'CompanyName': '青岛百年香传实业有限公司',
  'CompanyType': '有限责任公司(自然人独资)',
  'LegalPerson': '李 诗耀',
  'RegCapital': '1000.000000万',
  'companyTypeTags': ['独资企业', '有限责任公司'],
  'CompanyCode': '370220230030416',
  'CompanyAddress': '山东省青岛市市南区山东路6号甲1号楼3901室',
  'BusinessScope': '许可项目：食品销售；道路货物运输（ 不含危险货物）。（依法须经批准的项目，经相关部门批准后方可开展经营活动，具体经营项目以相关部门批准文件或许可证件为准）一般项目：货物进出口；办公用品销售；第一类医疗器械销售；教学用模型及教具销售；汽车零配件零售；互联网销售（除销售需要许可的商品）；无船承运业务；技术进出口；进出口代理；国内货物运输代理；物联网应用服务；装卸搬运；信息咨询服务（不含许可类信息咨询服务）；机械设备研发。（除依法须经批准的项目外，凭营业执照依法自主开展经营活动）',
  'Authority': '青岛市市南区 市场监督管理局',
  'CompanyStatus': '在营（开业）企业',
  'EstablishDate': '2014-11-12 00:00:00',
  'CreditNo': '91370220321511725F',
  'Province': '山东省',
  'OperationStartDate': '2014-11-12 00:00:00',
  'OperationEndDate': None,
  'IssueDate': '2024-07-18 00:00:00',
  'ProvinceCode': '370000',
  'City': '青岛市',
  'CityCode': '370200',
  'District': '市南区',
  'DistrictCode': '370202',
